In [1]:
import numpy as np
import time

In [2]:
# T1: dichotomy
def bisection(f, a, b, epsilon = 1e-4, max_iter = 100):  
    if a >= b:
        raise ValueError("b must be bigger than a")
        
    if f(a) * f(b) >= 0:  
        raise ValueError("f(a) * f(b) must be negative")  
  
    fa, fb = f(a), f(b)
    
    # if for iter_cnt in range(max_iter), although it may not report error out of for, it's not a good way
    iter_cnt = 0
    for _ in range(max_iter): 
        x = (a + b) / 2
        fx = f(x)
        error = np.abs(fx)
        if error < epsilon:
            break  
        
        if fa * fx < 0:
            b = x  
            fb = fx  
        else: 
            a = x  
            fa = fx

        iter_cnt += 1

    return x, error, iter_cnt


def f(x):
    return x**4/4 - 4*x**3/3 + 5*x**2/2 - 2*x

def df(x):
    return x**3 - 4*x**2 + 5*x - 2

root, _, _ = bisection(df, 0, 4)
print(f"{root}")

2.0


In [3]:
# T2: 0.618
def f(x):
    return np.exp(-x) + x**2

def goldencut(f, a, b, epsilon = 1e-4, max_iter = 100):
    if a >= b:
        raise ValueError("b must be bigger than a")

    t = (np.sqrt(5) - 1)/2
    iter_cnt = 0
    for _ in range(max_iter):
        error = b - a
        if error < epsilon:
            break
        
        x1 = a + (b - a)*(1 - t)
        x2 = a + (b - a)*t
        if f(x1) < f(x2):
            b = x2
        else:
            a = x1
        iter_cnt += 1
    return (a + b)/2, error, iter_cnt

minpoint, _, _ = goldencut(f, -2, 3)
print(f"{minpoint}")

0.35173474094718815


In [4]:
# T3: Newton
def f(x):
    return x**4 - 4*x**3 - 6*x**2 - 16*x + 4
    
def df(x):
    return 4*x**3 - 12*x**2 - 12*x - 16

def ddf(x):
    return 12*x**2 - 24*x - 12

def newton(f, x, df, epsilon = 1e-4, max_iter = 100):
    for iter_cnt in range(max_iter):
        fx = f(x)
        error = np.abs(fx)
        if error < epsilon:
            break
        
        x = x - fx/df(x)

    return x, error, iter_cnt

root, _, _ = newton(df, 2.5, ddf)
print(f"{root}")
root, _, _ = newton(df, 3, ddf)
print(f"{root}")

4.000000000017841
4.000000184090883


In [30]:
# T4: steepest descent, conjugate gradient, DFP
class OptimizationAlgorithm: 
    pass

def StepSelect_Newton(x, grad_f, Hesse_f, p, step, epsilon = 1e-7, max_iter = 5):
    for _ in range(max_iter):
        fstep = np.dot(grad_f(x + step*p), p)
        error = np.abs(fstep)
        if error < epsilon:
            break
        
        # dlambda f(x + lambda*p) = grad_f(x + lambda*p)*p
        step = step - fstep/np.dot(Hesse_f(x + step*p)@p, p)
        
    return step, x + step*p

def StepSelect_GoldenCut(f, a, b, epsilon = 1e-7, max_iter = 100):
    if a >= b:
        raise ValueError("b must be bigger than a")

    t = (np.sqrt(5) - 1)/2
    for _ in range(max_iter):
        error = b - a
        if error < epsilon:
            break
        
        x1 = a + (b - a)*(1 - t)
        x2 = a + (b - a)*t
        if f(x1) < f(x2):
            b = x2
        else:
            a = x1
    return (a + b)/2

# Steepest Descent
def SteepestDescent(f, grad_f, x, StepSelectMethod, Hesse_f, init_step = 0.1, epsilon = 1e-7, max_iter = 100):
    iter_cnt = 0
    for _ in range(max_iter):
        g = grad_f(x)
        p = -g
        if np.linalg.norm(g) < epsilon:
            break

        # find step = argmin_step f(x + step*p)
        if StepSelectMethod == "fixed":
            step = init_step
        elif StepSelectMethod == "Newton":
            step, _ = StepSelect_Newton(x, grad_f, Hesse_f, p, 0) 
        x = x + step*p
        iter_cnt += 1
    
    return x, f(x), iter_cnt


# Conjugate Gradient (FR)
class ConjugateGradient_FR():
    def __init__(self, x, f, epsilon = 1e-7, max_iter = 100):
        self.x = x
        self.f = f
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.step_select_methods = {  
            "fixed": {"init_step": 0.1},  
            "Newton": {"Hesse_f": None}, 
        }

    def optimize(self, grad_f, StepSelectMethod, **kwargs):
        if StepSelectMethod not in self.step_select_methods:  
            raise ValueError(f"Unknown step select method: {StepSelectMethod}")
        x = self.x
        f = self.f
        epsilon = self.epsilon
        max_iter = self.max_iter
        n = np.size(x)

        g = grad_f(x)
        iter_cnt = 0
        if np.linalg.norm(g) >= epsilon:
            if StepSelectMethod == "fixed":
                pass
            elif StepSelectMethod == "Newton":
                Hesse_f = kwargs["Hesse_f"]
            
            end = False
            for _ in range(max_iter):
                p = -g
                iter_in = 0
                while(1):
                    # find step = argmin_step f(x + step*p)
                    if StepSelectMethod == "fixed":
                        step = self.step_select_methods["fixed"]["init_step"]
                        x = x + step*p
                    elif StepSelectMethod == "Newton":
                        # dlambda f(x + lambda*p) = dot(grad_f(x + lambda*p), p)
                        # ddlambda f(x + lambda*p) = dot(Hesse_f(x + lambda*p)@p, p)
                        _, x = StepSelect_Newton(x, grad_f, Hesse_f, p, 0)
        
                    g_pre = g
                    g = grad_f(x)
        
                    if np.linalg.norm(g) < epsilon:
                        end = True
                        break
    
                    iter_in += 1
                    if iter_in == n:
                        break
                        
                    p = -g + (np.linalg.norm(g)/np.linalg.norm(g_pre))**2*p
                    
                if end == True:
                    break
                iter_cnt += 1
        return x, f(x), iter_cnt

# DFP
class DFP():
    def __init__(self, x, f, epsilon = 1e-6, max_iter = 100):
        self.x = x
        self.f = f
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.step_select_methods = {  
            "fixed": {"init_step": 0.1},  
            "Newton": {"Hesse_f": None},
            "GoldenCut": {"max_step": 1},
        }

    def optimize(self, grad_f, StepSelectMethod, **kwargs):
        if StepSelectMethod not in self.step_select_methods:  
            raise ValueError(f"Unknown step select method: {StepSelectMethod}")
        x = self.x
        f = self.f
        epsilon = self.epsilon
        max_iter = self.max_iter
        n = np.size(x)

        g = grad_f(x)
        iter_cnt = 0
        if np.linalg.norm(g) >= epsilon:
            if StepSelectMethod == "fixed":
                pass
            elif StepSelectMethod == "Newton":
                Hesse_f = kwargs["Hesse_f"]
            elif StepSelectMethod == "GoldenCut":
                if "max_step" in kwargs:
                    max_step = kwargs["max_step"]
                else:
                    max_step = self.step_select_methods["GoldenCut"]["max_step"]
            
            end = False
            for _ in range(max_iter):
                H = np.eye(n)
                p = -H@g
                iter_in = 0
                while(1):
                    # find step = argmin_step f(x + step*p)
                    if StepSelectMethod == "fixed":
                        step = self.step_select_methods["fixed"]["init_step"]
                        x = x + step*p
                    elif StepSelectMethod == "Newton":
                        # dlambda f(x + lambda*p) = dot(grad_f(x + lambda*p), p)
                        # ddlambda f(x + lambda*p) = dot(Hesse_f(x + lambda*p)@p, p)
                        step, x = StepSelect_Newton(x, grad_f, Hesse_f, p, 0)
                    elif StepSelectMethod == "GoldenCut":
                        def f_x_plus_lp(l):
                            return f(x + l*p)
                        step = StepSelect_GoldenCut(f_x_plus_lp, 0, max_step)
                        x = x + step*p
        
                    g_pre = g
                    g = grad_f(x)
                    
                    if np.linalg.norm(g) < epsilon:
                        end = True
                        break
    
                    iter_in += 1
                    if iter_in == n:
                        break
                        
                    delta_x = step*p
                    delta_g = g - g_pre
                    r = H@delta_g
                    # np.outer(x, x) is just delta_x[:, np.newaxis] @ delta_x[np.newaxis, :]
                    H += np.outer(delta_x, delta_x)/np.dot(delta_x, delta_g) - np.outer(r, r)/np.dot(delta_g, r)
                    
                if end == True:
                    break

                iter_cnt += 1
        return x, f(x), iter_cnt


def f(x):
    return 4*x[0]**2 + 4*x[1]**2 - 4*x[0]*x[1] - 12*x[1]

def grad_f(x):
    return np.array([8*x[0] - 4*x[1], 8*x[1] - 4*x[0] - 12])

def Hesse_f(x):
    return np.array([[8, -4], [-4, 8]])

# Attention: in python, vector and matrix is different.
# When use vector (such as variable x), it's better to use vector, x = np.array([1, 1])
# In matrix calculation, if it will not be confused, numpy will decide whether it is row or column vector automatically,
# otherwise, make a special announcement as `x[:, np.newaxis]`

start = time.time()
minpoint, fmin, itertime = SteepestDescent(f, grad_f, np.array([-1/2, 1]), "fixed", Hesse_f)
end = time.time()
print(f"Steepest Descent, Step select method: fixed")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

start = time.time()
minpoint, fmin, itertime = SteepestDescent(f, grad_f, np.array([-1/2, 1]), "Newton", Hesse_f)
end = time.time()
print(f"Steepest Descent, Step select method: Newton")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")



optimizer = ConjugateGradient_FR(np.array([-1/2, 1]), f)

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, "Newton", Hesse_f = Hesse_f)
end = time.time()
print(f"Conjugate Gradient (FR), Step select method: Newton")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")



optimizer = DFP(np.array([-1/2, 1]), f)

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, "GoldenCut")
end = time.time()
print(f"DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, "Newton", Hesse_f = Hesse_f)
end = time.time()
print(f"DFP, Step select method: Newton")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

Steepest Descent, Step select method: fixed
min point: [0.99999999 1.99999999], fmin: -12.0, iter time: 36
run time: 0.0009999275207519531

Steepest Descent, Step select method: Newton
min point: [0.99997635 1.99998423], fmin: -11.999999998260051, iter time: 100
run time: 0.0039997100830078125

Conjugate Gradient (FR), Step select method: Newton
min point: [1. 2.], fmin: -12.0, iter time: 0
run time: 0.0009999275207519531

DFP, Step select method: GoldenCut
min point: [0.99999996 1.99999989], fmin: -11.999999999999964, iter time: 20
run time: 0.017999887466430664

DFP, Step select method: Newton
min point: [0.99997635 1.99998423], fmin: -11.999999998260051, iter time: 100
run time: 0.0070002079010009766



In [28]:
# T5: Exterior & Interior Point
class ConstraintOptimization:
    pass


class ExteriorPoint:
    # Generally, accuracy of outer process should be lower then inner one, otherwise it can't converge
    def __init__(self, x, f, g, h, init_M = 1, c = 4, epsilon = 1e-3, max_iter = 100):
        self.x = x
        self.f = f
        self.g = g
        self.h = h
        self.init_M = init_M
        self.c = c
        self.epsilon = epsilon
        self.max_iter = max_iter

    def p(self, x, g, h):
        sum_g = 0
        sum_h = 0
        if g != None:
            gs = g(x)
            sum_g = np.sum(np.minimum(gs, np.zeros(np.size(gs)))**2)
        if h != None:
            sum_h = np.sum(h(x)**2)
        
        return sum_g + sum_h

    def grad_p(self, x, g, h, grad_g, grad_h):
        # grad_p = sum(g(x)*grad_g - |g(x)|*grad_g) + sum(2*h(x)*grad_h))
        #        = sum(2*min{g(x), 0}*grad_g) + sum(2*h(x)*grad_h))
        n = np.size(x)
        sum_g = np.zeros(n)
        sum_h = np.zeros(n)
        if g != None:
            gx = g(x)
            grad_gx = grad_g(x)
            # here use g(x) is a single-row matrix, we want each column of g(x) multiplies relative column of grad_g(x)
            sum_g = np.sum(gx*grad_gx - np.abs(gx)*grad_gx, axis = 1)
        if h != None:
            hx = h(x)
            grad_hx = grad_h(x)
            sum_h = np.sum(2*hx*grad_hx, axis = 1)
        
        return sum_g + sum_h
    
    def optimize(self, grad_f, grad_g, grad_h):
        x = self.x
        f = self.f
        g = self.g
        h = self.h 
        M = self.init_M
        c = self.c
        epsilon = self.epsilon
        max_iter = self.max_iter

        iter_cnt = 0
        for _ in range(max_iter):
            def f_plus_Mp(x):
                return f(x) + M*self.p(x, g, h)

            def grad_f_plus_Mp(x):
                return grad_f(x) + M*self.grad_p(x, g, h, grad_g, grad_h)
            
            optimizer = DFP(x, f_plus_Mp)
            x, f_plus_Mp_min, _ = optimizer.optimize(grad_f_plus_Mp, "GoldenCut", max_step = 0.1)
            # must judge error after once optimized, because if init point is in the field, error = 0
            error = np.abs(f_plus_Mp_min - f(x))
            if error < epsilon:
                break
            
            M = c*M
            iter_cnt += 1  
        return x, f(x), iter_cnt

class InteriorPoint:
    # accuracy should not be too high, otherwise the point will go out of the boundary
    def __init__(self, x, f, g, init_r = 1, c = 0.1, epsilon = 0.05, max_iter = 100):
        self.x = x
        self.f = f
        self.g = g
        self.init_r = init_r
        self.c = c
        self.epsilon = epsilon
        self.max_iter = max_iter
        self.punish_functions = {
            "reciprocal",
            "logarithm",
        }

    def B_r(self, x, g):
        sum_g = 0
        if g != None:
            sum_g = np.sum(1/g(x))
        
        return sum_g

    def grad_B_r(self, x, g, grad_g):
        # grad_B_r = sum(-grad_g(x)/[g(x)]^2)
        sum_g = np.zeros(np.size(x))
        if g != None:
            sum_g = np.sum(-1/g(x)**2*grad_g(x), axis = 1)
        
        return sum_g

    def B_l(self, x, g):
        sum_g = 0
        if g != None:
            sum_g = -np.sum(np.log(g(x)))
        return sum_g

    def grad_B_l(self, x, g, grad_g):
        # grad_B_l = -sum(grad_g(x)/g(x))
        sum_g = np.zeros(np.size(x))
        if g != None:
            sum_g = -np.sum(1/g(x)*grad_g(x), axis = 1)
        
        return sum_g
    
    def optimize(self, grad_f, grad_g, PunishFunction):
        x = self.x
        f = self.f
        g = self.g 
        r = self.init_r
        c = self.c
        epsilon = self.epsilon
        max_iter = self.max_iter

        if PunishFunction not in self.punish_functions:
            raise ValueError(f"Unknown punish function: {PunishFunction}")
        if PunishFunction == "reciprocal":
            B = self.B_r
            grad_B = self.grad_B_r
        elif PunishFunction == "logarithm":
            B = self.B_l
            grad_B = self.grad_B_l
        
        iter_cnt = 0
        for _ in range(max_iter):
            def f_plus_rB(x):
                return f(x) + r*B(x, g)

            def grad_f_plus_rB(x):
                return grad_f(x) + r*grad_B(x, g, grad_g)

            # observe the convergence process
            print(f"x:{x} f(x):{f(x)} r*B:{r*B(x, g)}")
            
            optimizer = DFP(x, f_plus_rB)
            x, f_plus_rB_min, _ = optimizer.optimize(grad_f_plus_rB, "GoldenCut", max_step = 0.01)
            # must judge error after once optimized, because if init point is in the field, error = 0
            error = np.abs(f_plus_rB_min - f(x))
            if error < epsilon:
                break
            
            r = c*r
            iter_cnt += 1
        return x, f(x), iter_cnt

In [17]:
# In this case, as the number of constraints can be single or multiple, constraints should represents as single-row matrix

# different dims of x by row, different constraints by column
def f(x):
    return x[0]**2 + x[1]**2

def grad_f(x):
    return np.array([2*x[0], 2*x[1]])

# remember to add an extra [] even if g(x) has only one formula,
# otherwise it will be regarded as vector but not matrix (eg. in numpy.sum, axis = 1 is not available)
def g(x):
    return np.array([[x[0] - 1]])

def grad_g(x):
    return np.array([[1], [0]])

print(f"function 1")
optimizer = ExteriorPoint(np.array([2, 2]), f, g, None)

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, grad_g, None)
end = time.time()
print(f"Exterior point, non-constraint method: DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

optimizer = InteriorPoint(np.array([2, 2]), f, g)

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, grad_g, "reciprocal")
end = time.time()
print(f"Interior point, punish function: reciprocal, non-constraint method: DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, grad_g, "logarithm")
end = time.time()
print(f"Interior point, punish function: logarithm, non-constraint method: DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

function 1
Exterior point, non-constraint method: DFP, Step select method: GoldenCut
min point: [9.99024390e-01 3.24611342e-07], fmin: 0.9980497319455207, iter time: 5
run time: 0.08800005912780762

x:[2 2] f(x):8 r*B:10.0
x:[ 2.43342767e+00 -3.45953032e-09] f(x):5.921570246498887 r*B:0.6976285011546657
x:[ 1.56519771e+00 -5.22243203e-16] f(x):2.449843873161911 r*B:0.17692923755796758
x:[5.61227777e-02 3.72204664e-17] f(x):0.003149766175120704 r*B:-0.010594598284168099
Interior point, punish function: reciprocal, non-constraint method: DFP, Step select method: GoldenCut
min point: [ 5.05089635e-03 -4.13672368e-19], fmin: 2.5511553937870017e-05, iter time: 3
run time: 0.03299999237060547

x:[2 2] f(x):8 r*B:-0.0
x:[2.79128769e+00 2.49881952e-07] f(x):7.791286966323492 r*B:-0.5829347409090372
x:[1.36602540e+00 1.17088329e-07] f(x):1.8660253974930239 r*B:0.10050525450338095
x:[1.04772255e+00 1.05032587e-07] f(x):1.0977225396904087 r*B:0.030423512673469995
Interior point, punish function: 

In [29]:
def f(x):
    return -x[0]*x[1]

def grad_f(x):
    return np.array([-x[1], -x[0]])

# g is a single-row matrix, but not vector
def g(x):
    return np.array([[-x[0] - x[1]**2 + 1, x[0] + x[1]]])

def grad_g(x):
    return np.array([[-1, 1], [-2*x[1], 1]])

print(f"function 2")
optimizer = ExteriorPoint(np.array([2, 2]), f, g, None)

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, grad_g, None)
end = time.time()
print(f"Exterior point, non-constraint method: DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

optimizer = InteriorPoint(np.array([0.5, 0.5]), f, g)

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, grad_g, "reciprocal")
end = time.time()
print(f"Interior point, punish function: reciprocal, non-constraint method: DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

start = time.time()
minpoint, fmin, itertime = optimizer.optimize(grad_f, grad_g, "logarithm")
end = time.time()
print(f"Interior point, punish function: logarithm, non-constraint method: DFP, Step select method: GoldenCut")
print(f"min point: {minpoint}, fmin: {fmin}, iter time: {itertime}")
print(f"run time: {end - start}\n")

function 2
Exterior point, non-constraint method: DFP, Step select method: GoldenCut
min point: [0.66744612 0.57765251], fmin: -0.3855519289214287, iter time: 4
run time: 0.9879999160766602

x:[0.5 0.5] f(x):-0.25 r*B:5.0
x:[0.19722759 0.45421414] f(x):-0.08958355938882287 r*B:0.32116096611014505
x:[0.38249179 0.4528998 ] f(x):-0.17323045618435354 r*B:0.03621932595352922
x:[0.57234874 0.54021587] f(x):-0.3091918691753714 r*B:0.00826161419513253
Interior point, punish function: reciprocal, non-constraint method: DFP, Step select method: GoldenCut
min point: [0.63732836 0.56621404], fmin: -0.36086426387137066, iter time: 3
run time: 0.6490001678466797

x:[0.5 0.5] f(x):-0.25 r*B:1.3862943611198906
x:[0.2608292  0.44877479] f(x):-0.11705356828713369 r*B:0.09633688469896051
x:[0.5511195  0.53431868] f(x):-0.29447344301899026 r*B:0.017296679595515816
Interior point, punish function: logarithm, non-constraint method: DFP, Step select method: GoldenCut
min point: [0.65332398 0.57393144], fmin